Note: we did not use China dataset for the project :(

In [ ]:
# get the data loader
from google.colab import files
uploaded = files.upload()

Saving data_loading.py to data_loading.py


In [ ]:
from data_loading import get_data_BIG_WGAN # use the data loader function to get images for all types of setups
(poland_images, poland_labels), (china_images, china_labels), (egypt_images, egypt_labels), (egypt_mixed_images_1180, egypt_mixed_labels_1180), (egypt_mixed_images_2360, egypt_mixed_labels_2360) = get_data_BIG_WGAN()

downloading a file with egypt 1180 samples (Main + BIG-WGAN) and poland, china


Downloading...
From (original): https://drive.google.com/uc?id=1PY29AhKzMo_msc0-8nc0oXnyWQHpbjWO
From (redirected): https://drive.google.com/uc?id=1PY29AhKzMo_msc0-8nc0oXnyWQHpbjWO&confirm=t&uuid=b8edf0ae-22d0-45e9-a113-44c9e12fabe7
To: /content/data_zipped
100%|██████████| 47.6M/47.6M [00:00<00:00, 73.9MB/s]


Poland: images (256, 128, 128), labels (256,)
China: images (635, 128, 128), labels (635,)
Egypt: images (780, 128, 128), labels (780,)
unbalanced, unshuffled,
0 for malignant, 1 for benign,
Poland label distribution:
  Label 0: 102
  Label 1: 154

China label distribution:
  Label 0: 435
  Label 1: 200

Egypt label distribution:
  Label 0: 210
  Label 1: 570

Egypt mixed with GAN:
  Label 0: 410
  Label 1: 770

Egypt mixed with GAN and FLIPPED:
  Label 0: 820
  Label 1: 1540
returns poland images(TEST THIS), china images, egypt images, egypt mixed images 1180(TRAIN THIS), egypt mixed images 2360(TRAIN THIS)


In [ ]:
# now look at the content (data_unzipped), dont use the loader variable
# data is here /content/data_unzipped/Datasets
import numpy as np

# generated images trained on the real egypt ,
#generated_egypt_images = np.load('/content/data_unzipped/Datasets/GAN_images_rdy/gan_images_128x128_gray_shuffled.npy')
#generated_egypt_labels = np.load('/content/data_unzipped/Datasets/GAN_images_rdy/gan_labels_shuffled.npy')
# real images of egypt
real_egypt_images = np.load('/content/data_unzipped/Datasets/Egypt Hospital - Breast Ultrasound/Egypt Hospital 128x128/Egypt Hospital_US_Breast_Images.npy')
real_egypt_labels = np.load('/content/data_unzipped/Datasets/Egypt Hospital - Breast Ultrasound/Egypt Hospital 128x128/Egypt Hospital_US_Breast_Labels.npy')
# get poland for tests
poland_images = np.load('/content/data_unzipped/Datasets/Polish Hospital - Breast Ultrasound 128x128/Poland_Medical_Centers_US_Breast_Images.npy')
poland_labels = np.load('/content/data_unzipped/Datasets/Polish Hospital - Breast Ultrasound 128x128/Poland_Medical_Centers_US_Breast_Labels.npy')

In [ ]:
# turn to floats
#generated_egypt_images = generated_egypt_images.astype(np.float32)
#generated_egypt_labels = generated_egypt_labels.astype(np.float32)
poland_images = poland_images.astype(np.float32)
poland_labels = poland_labels.astype(np.float32)
real_egypt_images = real_egypt_images.astype(np.float32)
real_egypt_labels = real_egypt_labels.astype(np.float32)

In [ ]:
#generated_egypt_images = (generated_egypt_images / 127.5) - 1
real_egypt_images = (real_egypt_images / 127.5) - 1
poland_images = (poland_images / 127.5) - 1

In [ ]:
# this block is MEGA IMPORTANT, her is where you set everything about ratios
#generated_egypt_images = generated_egypt_images[:400] # here is where you change 0,25,50,75,100
#generated_egypt_labels = generated_egypt_labels[:400]
real_egypt_images = real_egypt_images[:780] # these two are always a hunderd
real_egypt_labels = real_egypt_labels[:780]
poland_images = poland_images[:256] # these are always a hundred
poland_labels = poland_labels[:256]

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torchvision.models as models

In [ ]:
egypt_mixed_images_1180 = (egypt_mixed_images_1180 / 127.5) - 1
egypt_mixed_images_2360 = (egypt_mixed_images_2360 / 127.5) - 1

In [ ]:
#lazy reassignment of inputs to floats
X_train = egypt_mixed_images_2360.astype(np.float32)
Y_train = egypt_mixed_labels_2360.astype(np.float32)
X_val = poland_images.astype(np.float32)
Y_val = poland_labels.astype(np.float32)

In [ ]:
X_train.shape, X_val.shape, Y_train.shape, Y_val.shape

((2360, 128, 128), (256, 128, 128), (2360,), (256,))

In [ ]:
# tensorize for the network
X_train = torch.tensor(X_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
Y_train = torch.tensor(Y_train, dtype=torch.float32)
Y_val = torch.tensor(Y_val, dtype=torch.float32)

print(X_train.shape, X_val.shape, Y_train.shape, Y_val.shape)
# get the batch dim out ofthere
X_train = X_train.unsqueeze(1)
X_val = X_val.unsqueeze(1)

# The Resnet really wanted a long type, idk why
Y_train = Y_train.long()
Y_val = Y_val.long()

train_ds = TensorDataset(X_train, Y_train)
test_ds = TensorDataset(X_val, Y_val)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=32)


torch.Size([2360, 128, 128]) torch.Size([256, 128, 128]) torch.Size([2360]) torch.Size([256])


In [ ]:
print(X_train.shape, X_val.shape, Y_train.shape, Y_val.shape)

torch.Size([2360, 1, 128, 128]) torch.Size([256, 1, 128, 128]) torch.Size([2360]) torch.Size([256])


In [ ]:
import torchvision.models as models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#get basic pre-trained resnet
model = models.resnet50(pretrained=True)

# greyscale, so one channel in
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

# last layer, fully-connected into binary classification
model.fc = nn.Linear(model.fc.in_features, 1)

model = model.to(device)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
import torch.nn as nn
import torch.optim as optim
# BCE and adam, super standard
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)


In [ ]:
# simple traing function
def train(model, train_loader, optimizer, criterion, device, epochs=2):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}")


In [ ]:
# test, take average
def test(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            _, pred = out.max(1)
            correct += (pred == y).sum().item()
            total += y.size(0)
    acc = 100 * correct / total
    return acc


In [ ]:
# call train
train(model, train_loader, optimizer, criterion, device, epochs=20)

# call test
test(model, test_loader, device)


Epoch 1/20, Loss: 0.6867
Epoch 2/20, Loss: 0.3878
Epoch 3/20, Loss: 0.3306
Epoch 4/20, Loss: 0.2964
Epoch 5/20, Loss: 0.2963
Epoch 6/20, Loss: 0.2754
Epoch 7/20, Loss: 0.2808
Epoch 8/20, Loss: 0.2932
Epoch 9/20, Loss: 0.2729
Epoch 10/20, Loss: 0.2717
Epoch 11/20, Loss: 0.2725
Epoch 12/20, Loss: 0.2611
Epoch 13/20, Loss: 0.2520
Epoch 14/20, Loss: 0.2532
Epoch 15/20, Loss: 0.2492
Epoch 16/20, Loss: 0.2464
Epoch 17/20, Loss: 0.2538
Epoch 18/20, Loss: 0.2495
Epoch 19/20, Loss: 0.2476
Epoch 20/20, Loss: 0.2554


65.625

Big cross domain tests:  
full egypt on poland 61.3%  
egypt 1180 on poland 60.15%  
egypt 2360 on poland 65.62%  


Train all data on the same domain (wild fluctation 73-87%.., we'll use Drew's results of average accuracy from the google sheets)    
Train 100 real egypt 100 fake egypt Test Poland 66%  
Train 100 real egypt 75 fake egypt Test Poland 64%   
Train 100 real egypt 50 fake egypt Test Poland 63%  
Train 100 real egypt 25 fake egypt Test Poland 66%   
Train 100 real egypt 0 fake egypt Test Poland  65%  
Train all real egypt test on all poland  62.5%  
Train all real egypt with all fake egypt on all poland 60.55%  

